In [1]:
import pandas as pd
from pathlib import Path
from collections import Counter
import os
import difflib
import math

In [2]:
DATA_PATH = Path("/home/pengfei/data/pdtb_v2/data/raw/wsj/")

In [3]:
pdtb2 = pd.read_csv("../pdtb2/pdtb2.csv")

/home/pengfei/miniconda3/envs/stanfordnlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


in terms of segment number:
single, double, trible = (38584, 1948, 51)

how many distance away from double span:


    Counter(a).most_common()
    
    
    all of them is within 5

How many distances away:
   
   
   Counter({0: 37713,
         1: 2598,
         2: 193,
         6: 7,
         12: 1,
         9: 3,
         3: 50,
         5: 5,
         18: 2,
         4: 18,
         13: 1,
         8: 4,
         15: 1,
         62: 1,
         19: 1,
         10: 1,
         16: 1})

need correction distance:
    [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 18, 19, 62]

In [5]:
# write functions

In [6]:
def _get_files(data_path=DATA_PATH):
    """Returns (list): [('00', 'wsj_0000'), ..., ('24', 'wsj_2400']"""
    sections = os.listdir(data_path)
    ret = []
    for sec in sections:
        for filename in os.listdir(data_path/sec):
            ret.append((sec, filename))
    return ret

def get_batch(section, filenumber):
    """
    Args: 
            section(int)
            filenumber(int)
    Returns:
            (list[int]): list of index with all the file with the same format in that batch"""
    return pdtb2.index[(pdtb2['Section'] == section) & (pdtb2.FileNumber == filenumber)].tolist()


def _get_arg(char_span_list, rawtext):
    arg = ''
    for span in char_span_list:
        arg += rawtext[int(span[0])-2:int(span[1])-2]
    return arg

def _get_span_list(span):
    """
    Args:
            span(str): "34..96;97..101"
    Returns:
            ret(list[list[str]]):
            if one arg has one segment: [[34,96]]
            if one arg has two segment: example [[34, 96], [97, 101]]
    """
    if span == '':
        return []
    spans = span.split(';')
    return [[int(k) for k in o.split('..')] for o in spans ]

def check_number_of_errors():
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                if len(arg1_char_span_list) == 1:
                    single += 1
                elif len(arg1_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1
    print("error numbers for arg1  single:", single, " double: ", double, " triple: ", triple)
    count = 0
    single = 0
    double = 0
    triple = 0
    files =  _get_files()
    for (section, filename) in files:
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg2
            arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
            arg2_char_span_list = _get_span_list(arg2_char_span_string)
            arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
            if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
                if len(arg2_char_span_list) == 1:
                    single += 1
                elif len(arg2_char_span_list) == 2:
                    double += 1
                else:
                    triple += 1
    print("error numbers for arg2  single:", single, " double: ", double, " triple: ", triple)
    
def get_length(list):
    length = 0
    for o in list:
        length += o[1] - o[0]
    return length


def get_start_end(span_list, rawtext, tolerance_size):
    tolerance_size += 1
    start = span_list[0][0]
    end = span_list[len(span_list)-1][1]
    if start-tolerance_size<0:
        start = 0
    else:
        start -= tolerance_size
    if end + tolerance_size>len(rawtext)-1:
        end = len(rawtext) -1
    else:
        end += len(rawtext)+tolerance_size
    
    return start, end

def correct_span(span_list, arg_rawtext, rawtext):
    """
    Args: 
            span_list[list[[int, int], ...]]
            arg_rawtext(str)
            rawtext(str): doc raw text
    Returns: 
            corrected_span(str): [34, 39]
    """
    corrected_span = ''
    tolerance_size = abs(get_length(span_list) - len(arg_rawtext))
    start, end = get_start_end(span_list, rawtext, tolerance_size)

    s = difflib.SequenceMatcher(None, rawtext[start: end], arg_rawtext)
    for i, j, n in s.get_matching_blocks():
        if n!=0:
            corrected_span += f'{start+i+2}..{start+i+n+2};' # here is a offset of 2 char we need to consider
    corrected_span = corrected_span[:-1]
    return corrected_span

def sanity_check(corrected_span, rawtext, arg_rawtext):
    """Args: 
            corrected_span(str)
            rawtext(str): doc string
            arg_rawtext(str): gold argument string
    Returns: 
            (boolean): the corrected span is corrected"""
    span_list_corrected = _get_span_list(corrected_span)
    return _get_arg(span_list_corrected, rawtext) == arg_rawtext

In [ ]:
def correct_docs():
    files =  _get_files()
    count = 0
    for (section, filename) in files:
        print('section:', section, ' files: ', filename)
        with open(DATA_PATH/section/filename, encoding='latin-1') as f:
            rawtext = f.read()
            rawtext = rawtext.replace('\n', '')
        section = int(section)
        filenumber = int(filename[-2:])
        batch = get_batch(section, filenumber)
        for i in batch:
            # for arg1
            arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
            arg1_char_span_list = _get_span_list(arg1_char_span_string)
            arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
            if _get_arg(arg1_char_span_list, rawtext) != arg1_rawtext:
                corrected_span = correct_span(arg1_char_span_list, arg1_rawtext, rawtext)
                arg1_char_span_list_corrected = _get_span_list(corrected_span)

                count += 1
                pdtb2.loc[i, 'Arg1_SpanList'] = corrected_span

            # for arg2
            arg2_char_span_string = pdtb2.loc[i, 'Arg2_SpanList']
            arg2_char_span_list = _get_span_list(arg2_char_span_string)
            arg2_rawtext = pdtb2.loc[i, 'Arg2_RawText']
            if _get_arg(arg2_char_span_list, rawtext) != arg2_rawtext:
                corrected_span = correct_span(arg2_char_span_list, arg2_rawtext, rawtext)

                count += 1
                pdtb2.loc[i, 'Arg2_SpanList'] = corrected_span
    return pdtb2

section: 07  files:  wsj_0793
section: 07  files:  wsj_0739
section: 07  files:  wsj_0712
section: 07  files:  wsj_0777
section: 07  files:  wsj_0709
section: 07  files:  wsj_0797
section: 07  files:  wsj_0799
section: 07  files:  wsj_0751
section: 07  files:  wsj_0781
section: 07  files:  wsj_0782
section: 07  files:  wsj_0740
section: 07  files:  wsj_0732
section: 07  files:  wsj_0728
section: 07  files:  wsj_0766
section: 07  files:  wsj_0772
section: 07  files:  wsj_0748
section: 07  files:  wsj_0764
section: 07  files:  wsj_0710
section: 07  files:  wsj_0767
section: 07  files:  wsj_0705
section: 07  files:  wsj_0750
section: 07  files:  wsj_0789
section: 07  files:  wsj_0708
section: 07  files:  wsj_0795
section: 07  files:  wsj_0761
section: 07  files:  wsj_0791
section: 07  files:  wsj_0734
section: 07  files:  wsj_0759
section: 07  files:  wsj_0701
section: 07  files:  wsj_0738
section: 07  files:  wsj_0737
section: 07  files:  wsj_0755
section: 07  files:  wsj_0742
section: 0

section: 06  files:  wsj_0603
section: 06  files:  wsj_0654
section: 06  files:  wsj_0620
section: 06  files:  wsj_0693
section: 06  files:  wsj_0688
section: 06  files:  wsj_0605
section: 06  files:  wsj_0660
section: 06  files:  wsj_0642
section: 06  files:  wsj_0699
section: 06  files:  wsj_0679
section: 06  files:  wsj_0610
section: 06  files:  wsj_0668
section: 06  files:  wsj_0617
section: 06  files:  wsj_0635
section: 06  files:  wsj_0633
section: 06  files:  wsj_0623
section: 06  files:  wsj_0689
section: 06  files:  wsj_0669
section: 06  files:  wsj_0696
section: 06  files:  wsj_0607
section: 06  files:  wsj_0680
section: 06  files:  wsj_0678
section: 06  files:  wsj_0670
section: 06  files:  wsj_0652
section: 02  files:  wsj_0262
section: 02  files:  wsj_0225
section: 02  files:  wsj_0275
section: 02  files:  wsj_0220
section: 02  files:  wsj_0252
section: 02  files:  wsj_0276
section: 02  files:  wsj_0202
section: 02  files:  wsj_0264
section: 02  files:  wsj_0289
section: 0

section: 01  files:  wsj_0179
section: 01  files:  wsj_0160
section: 01  files:  wsj_0128
section: 01  files:  wsj_0170
section: 01  files:  wsj_0130
section: 01  files:  wsj_0198
section: 01  files:  wsj_0113
section: 01  files:  wsj_0119
section: 01  files:  wsj_0189
section: 01  files:  wsj_0161
section: 01  files:  wsj_0135
section: 01  files:  wsj_0172
section: 01  files:  wsj_0148
section: 01  files:  wsj_0142
section: 01  files:  wsj_0162
section: 01  files:  wsj_0151
section: 01  files:  wsj_0137
section: 01  files:  wsj_0182
section: 01  files:  wsj_0174
section: 01  files:  wsj_0118
section: 01  files:  wsj_0196
section: 01  files:  wsj_0175
section: 01  files:  wsj_0199
section: 01  files:  wsj_0168
section: 01  files:  wsj_0173
section: 01  files:  wsj_0167
section: 01  files:  wsj_0184
section: 01  files:  wsj_0157
section: 01  files:  wsj_0100
section: 01  files:  wsj_0144
section: 01  files:  wsj_0103
section: 01  files:  wsj_0177
section: 01  files:  wsj_0150
section: 0

section: 09  files:  wsj_0912
section: 09  files:  wsj_0929
section: 09  files:  wsj_0967
section: 09  files:  wsj_0941
section: 09  files:  wsj_0918
section: 09  files:  wsj_0983
section: 09  files:  wsj_0992
section: 09  files:  wsj_0970
section: 09  files:  wsj_0924
section: 09  files:  wsj_0940
section: 09  files:  wsj_0972
section: 09  files:  wsj_0944
section: 09  files:  wsj_0952
section: 09  files:  wsj_0969
section: 09  files:  wsj_0968
section: 09  files:  wsj_0920
section: 09  files:  wsj_0932
section: 09  files:  wsj_0947
section: 09  files:  wsj_0978
section: 09  files:  wsj_0995
section: 09  files:  wsj_0939
section: 09  files:  wsj_0931
section: 09  files:  wsj_0993
section: 09  files:  wsj_0987
section: 09  files:  wsj_0965
section: 09  files:  wsj_0928
section: 09  files:  wsj_0976
section: 09  files:  wsj_0914
section: 09  files:  wsj_0980
section: 09  files:  wsj_0909
section: 09  files:  wsj_0977
section: 09  files:  wsj_0917
section: 09  files:  wsj_0901
section: 0

section: 17  files:  wsj_1707
section: 17  files:  wsj_1793
section: 17  files:  wsj_1752
section: 17  files:  wsj_1718
section: 17  files:  wsj_1784
section: 17  files:  wsj_1739
section: 17  files:  wsj_1728
section: 17  files:  wsj_1751
section: 17  files:  wsj_1769
section: 17  files:  wsj_1716
section: 17  files:  wsj_1740
section: 17  files:  wsj_1788
section: 17  files:  wsj_1704
section: 17  files:  wsj_1724
section: 17  files:  wsj_1749
section: 17  files:  wsj_1711
section: 17  files:  wsj_1732
section: 17  files:  wsj_1743
section: 17  files:  wsj_1715
section: 17  files:  wsj_1735
section: 17  files:  wsj_1754
section: 17  files:  wsj_1790
section: 17  files:  wsj_1753
section: 17  files:  wsj_1730
section: 17  files:  wsj_1746
section: 17  files:  wsj_1761
section: 17  files:  wsj_1767
section: 17  files:  wsj_1768
section: 17  files:  wsj_1710
section: 17  files:  wsj_1792
section: 17  files:  wsj_1773
section: 17  files:  wsj_1796
section: 17  files:  wsj_1779
section: 1

section: 12  files:  wsj_1219
section: 12  files:  wsj_1233
section: 12  files:  wsj_1257
section: 12  files:  wsj_1203
section: 12  files:  wsj_1206
section: 12  files:  wsj_1229
section: 12  files:  wsj_1294
section: 12  files:  wsj_1282
section: 12  files:  wsj_1227
section: 12  files:  wsj_1243
section: 12  files:  wsj_1222
section: 12  files:  wsj_1271
section: 12  files:  wsj_1278
section: 12  files:  wsj_1256
section: 12  files:  wsj_1286
section: 12  files:  wsj_1279
section: 12  files:  wsj_1211
section: 12  files:  wsj_1246
section: 12  files:  wsj_1240
section: 12  files:  wsj_1200
section: 12  files:  wsj_1250
section: 12  files:  wsj_1273
section: 12  files:  wsj_1230
section: 12  files:  wsj_1262
section: 12  files:  wsj_1277
section: 12  files:  wsj_1213
section: 00  files:  wsj_0076
section: 00  files:  wsj_0063
section: 00  files:  wsj_0069
section: 00  files:  wsj_0033
section: 00  files:  wsj_0095
section: 00  files:  wsj_0038
section: 00  files:  wsj_0051
section: 0

section: 11  files:  wsj_1166
section: 11  files:  wsj_1154
section: 11  files:  wsj_1102
section: 11  files:  wsj_1186
section: 11  files:  wsj_1116
section: 11  files:  wsj_1156
section: 11  files:  wsj_1163
section: 11  files:  wsj_1179
section: 11  files:  wsj_1171
section: 11  files:  wsj_1180
section: 11  files:  wsj_1136
section: 11  files:  wsj_1131
section: 11  files:  wsj_1174
section: 11  files:  wsj_1109
section: 11  files:  wsj_1178
section: 11  files:  wsj_1193
section: 11  files:  wsj_1161
section: 11  files:  wsj_1195
section: 11  files:  wsj_1170
section: 11  files:  wsj_1189
section: 11  files:  wsj_1175
section: 11  files:  wsj_1127
section: 11  files:  wsj_1184
section: 11  files:  wsj_1103
section: 11  files:  wsj_1110
section: 11  files:  wsj_1149
section: 11  files:  wsj_1140
section: 11  files:  wsj_1122
section: 11  files:  wsj_1100
section: 11  files:  wsj_1194
section: 11  files:  wsj_1181
section: 11  files:  wsj_1159
section: 11  files:  wsj_1148
section: 1

In [74]:
pdtb2.to_csv('pdtb2.csv')